- **<font color=blue size=3> 1.GFED2WRF-Chem software was developed by Professor Fan Qi's team at Sun Yat-sen University: Yinbao Jin, Yuqi Zhu and others.</font>**<br>
- **<font color=blue size=3> 2.email:ybjin3@mail2.sysu.edu.cn</font>**<br>

**<center><font color=red size=5>Importing python packages</font></center>**

In [29]:
import xarray as xr
import numpy as np
import h5py
import xesmf as xe
import pandas as pd
import glob
import os
from netCDF4 import Dataset
from shutil import copyfile
from wrf import getvar, ALL_TIMES,latlon_coords,to_np

**<center><font color=red size=5>Functions</font></center>**

In [30]:
def Dealwith_onlyday(MM,DD):
    newnum = str(int(DD))
    Day_Frac_Path = 'emissions/'+MM+'/daily_fraction/day_'+newnum
    Day_Frac = f[Day_Frac_Path]
    Emission_DM = 'emissions/'+MM+'/DM'
    Mar_DM_2010 = f[Emission_DM] #读取变量（720,1440）
    Mar_DM_2010 = Mar_DM_2010[:]
    GFED_Dataset = np.zeros([24,33,GFED_latitude.shape[0],GFED_latitude.shape[1]])
    for i in np.arange(1,25):## gas unit from WRF-Chem: mol km^-2 hr^-1
        for j in np.arange(1,30):
            temp = np.zeros([GFED_latitude.shape[0],GFED_latitude.shape[1]])
            for k in np.arange(1,7):
                Classification_Contri_Path = 'emissions/'+MM+'/partitioning/DM_'+ classification[k-1]
                Classification_Contri = f[Classification_Contri_Path]
                Times_Path = 'emissions/'+MM+'/diurnal_cycle/'+Times[i-1]
                H = f[Times_Path]
                # g species m^-2  month^-1 -> mol km^-2 hr^-1
                temp += (0.33*10**6)*(1/GFED_Factor.MW[j-1])*GFED_Factors[j-1,k-1]*(Day_Frac)*(H[:])*Mar_DM_2010*Classification_Contri[:]
            GFED_Dataset[i-1,j-1,:,:] = temp
    for i in np.arange(1,25): # aerosol unit from WRF-Chem: ug m^-2 s^-1
        for j in np.arange(30,34):
            temp = np.zeros([GFED_latitude.shape[0],GFED_latitude.shape[1]])
            for k in np.arange(1,7):
                Classification_Contri_Path = 'emissions/'+MM+'/partitioning/DM_'+ classification[k-1]
                Classification_Contri = f[Classification_Contri_Path]
                Times_Path = 'emissions/'+MM+'/diurnal_cycle/'+Times[i-1]
                H = f[Times_Path]
                # g species m^-2  month^-1 -> ug m^-2 s^-1
                temp +=(1/3600)*(0.33*10**6)*(1/GFED_Factor.MW[j-1])*GFED_Factors[j-1,k-1]*(Day_Frac)*(H[:])*Mar_DM_2010*Classification_Contri[:]
            GFED_Dataset[i-1,j-1,:,:] = temp
    return regridder(GFED_Dataset)
def Write_Data_to_WRFDomain(data,HH):
    Classifi_WRF = ["ebu_in_co", "ebu_in_no", "ebu_in_so2", "ebu_in_bigalk", "ebu_in_bigene", "ebu_in_c2h4",
               "ebu_in_c2h5oh", "ebu_in_c2h6", "ebu_in_c3h8", "ebu_in_c3h6", "ebu_in_ch2o", "ebu_in_ch3cho",
               "ebu_in_ch3coch3", "ebu_in_ch3oh", "ebu_in_mek", "ebu_in_toluene", "ebu_in_nh3", "ebu_in_no2",
               "ebu_in_open", "ebu_in_c10h16", "ebu_in_ch3cooh", "ebu_in_cres", "ebu_in_glyald", "ebu_in_mgly",
               "ebu_in_gly", "ebu_in_acetol", "ebu_in_isop", "ebu_in_macr", "ebu_in_mvk", "ebu_in_oc",
               "ebu_in_bc", "ebu_in_pm10", "ebu_in_pm25"]
    copyfile(Input+filename,Output+'/'+filename)
    GFED_Data = Dataset(Output+'/'+filename,'a')
    for j in range(1,34):
        GFED_Data.variables[Classifi_WRF[j-1]][0,0,:,:] = data[int(HH),j-1,:,:]
    GFED_Data.close()

**<center><font color=red size=5>Process</font></center>**

In [33]:
# Need to modify the place
Input = './FINN1.5/'
Output = 'new_GFED'
GFED_raw_data = 'GFED4.1s_2019_beta.hdf5'
wrf_domain = 'wrfinput_d01'
EF = 'GFED_Factors.xlsx'
#--------------------------------------------------------------------------------------------------------
# 检查文件夹是否存在
if os.path.exists(Output):
    print("必要文件夹已存在，可以进行文件处理。")
else:
    # 创建新文件夹
    os.mkdir(Output)
    print("1) 已创建新文件夹：", Output)
# list data
FileName = glob.glob(Input+'wrf*')
numfiles = len(FileName)
print("\033[34;1;5m" +"2) 总共 "+ str(numfiles) + " 新的GFED文件将会产生！它们名称如下：\033[0m")
# open HDF5
f = h5py.File(GFED_raw_data, 'r')
# read lat and lon 
longitude_dataset = f['lon']
GFED_longitude = longitude_dataset[:]
latitude_dataset = f['lat']
GFED_latitude = latitude_dataset[:]
# open wrfinput_d01
WRF_Data = Dataset(wrf_domain)
HGT = getvar(WRF_Data,'HGT',timeidx=ALL_TIMES)#指定时间或者timeidx=ALL_TIMES
lats, lons = latlon_coords(HGT)
#Interpolation method
grid_in = {"lon": GFED_longitude, "lat": GFED_latitude}
grid_out = {"lon": lons, "lat": lats}
regridder = xe.Regridder(grid_in, grid_out, "bilinear") 
#EF
GFED_Factor = pd.read_excel(EF,skiprows=0,
                            usecols=[0,1,2,3,4,5,6,7],
                            names=['species','MW','SAVA','BORF','TEMF','DEFO','PEAT','AGRI'])
new_GFED_Factor = GFED_Factor.values
GFED_Factors=new_GFED_Factor[:,2:8]
classification= ["SAVA","BORF","TEMF","DEFO","PEAT","AGRI"]
Times = ['UTC_0-3h','UTC_0-3h','UTC_0-3h','UTC_3-6h','UTC_3-6h','UTC_3-6h','UTC_6-9h','UTC_6-9h',
    'UTC_6-9h','UTC_9-12h','UTC_9-12h','UTC_9-12h','UTC_12-15h','UTC_12-15h','UTC_12-15h','UTC_15-18h',
    'UTC_15-18h','UTC_15-18h','UTC_18-21h','UTC_18-21h','UTC_18-21h','UTC_21-24h','UTC_21-24h','UTC_21-24h']
#----------------------------------------------------------------------------------------------------------
for i in range(numfiles):
    file_path = FileName[i]
    file_name = os.path.basename(file_path)
    date_parts = file_name.split('_')[2].split('-')
    MM = date_parts[1]
    DD = date_parts[2]
    HH = file_name.split('_')[3]
    print("处理第 "+ str(i+1)+" 个文件 "+filename,"month: "+MM,"day: "+DD,"hour: "+HH)
    GFEDdata = Dealwith_onlyday(MM,DD)
    Write_Data_to_WRFDomain(GFEDdata,HH)

已创建新文件夹： new_GFED
总共 2 new emission inventory data will be generated !
处理第 1 个文件 wrffirechemi_d01_2019-02-26_01_00_00.nc month: 02 day: 26
处理第 2 个文件 wrffirechemi_d01_2019-02-26_01_00_00.nc month: 02 day: 26
